# Stage Three - Data Preparation

This is the stage of the project where you decide on the data that you're going to use for analysis. The criteria you might use to make this decision include the relevance of the data to your data mining goals, the quality of the data, and also technical constraints such as limits on data volume or data types. Note that data selection covers selection of attributes (columns) as well as selection of records (rows) in a table.

Table of Content:
0. Loading Modules, Functions and Dataset
1. Load the Data
2. Clean The Data
3. Merge Data sets
4. Clean up Merged Data Set
5. Add Addiional Predictors
6. Create and Clean Target Data set


## Assumptions
1. Calculated Fields
*	 Calculate the 7 day and 28-day heart rate average (
*	 Calculate 7 day and 28 day 1, 2, 3 and above 3 std dev from average
*	 Calculate if the 7-day average is above, at or below the 28-day average
2.	Target Data set
*	Change the Current Status in to 4
*	How to add to the Merges Data set on a daily Basis
*	Weeks are Wednesday to Tuesday
*	Survey done on Tuesday but may be turned in after
> If date is a Tuesday , then survey date = date
>
> If date not Tuesday then survey date = prior Tuesday’s date
* PSS score
> 0-13 low stress
>
> 14-26 Mod
>
> 26-40 High Stress
3.	Drop any dates not in date rage of Weekly Survey data
* Survey dates are 5/19/2020 to 4/10/2020
> Only use daily records from 5/13/2020 – 4/6/2021
* Weeks are Wednesday to Tuesday
* Survey done on Tuesday but may be turned in after


## Step 0: Loading Modules and Dataset

This section is to load and modules and the original dataset from a CSV file into a dataframe

### Modules

In [ ]:
# importing  packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re

## importing datetime class
from datetime import datetime, timedelta

from sklearn.preprocessing import OneHotEncoder

###Set other defaults

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

### Functions
This section is where all the functions are


#### Functions - Weekly Survey dataset
* statusconditions - Creates new target field
* getTuesdaydate   - Creates new standard survey date field

In [ ]:
def statusconditions(x):
    '''
    This function looks multiple condition of the passed values and returns 
    the new value 
    Input 
    - x -  column name of field to evaluate
    Output
    - Returns decoded value or NAN if none match
    '''
    cs_answ1 = 'About the same'	                           
    cs_answ2 = 'Better than usual'	                         
    cs_answ3 = 'I felt over-stressed and/or under-recovered'
    cs_answ4 = 'I had a flare up of a chronic condition'	   
    cs_answ5 = 'I had a new onset of illness'	             
    cs_answ6 = 'I had increased pain or an injury'	         
    cs_answ7 = 'Significantly Worse, in distress'	 

    # ns_answ1 = 'Same'
    # ns_answ2 = 'Better'
    # ns_answ3 = 'Stress'
    # ns_answ4 = 'Unwell'
    
    ns_answ1 = 1
    ns_answ2 = 2
    ns_answ3 = 3
    ns_answ4 = 4


    if x == cs_answ1:
        return ns_answ1
    elif x == cs_answ2:
        return ns_answ2
    elif x == cs_answ3:
        return ns_answ3
    elif x == cs_answ4:
        return ns_answ4
    elif x == cs_answ5:
        return ns_answ4
    elif x == cs_answ6:
        return ns_answ4
    elif x == cs_answ7:
        return ns_answ4
    else:
        return np.Nan

# func = np.vectorize(statusconditions)
# df_survey['Status'] = func(df_survey['Current Status'])


def getTuesdaydate(origdate,dir=0):
    '''
    This function takes a datetime values and return a date 
    If the date is a Tuesday, it returns the past date
    else it returns the prior Tuesday's date 
    Input -
      origdate - datetime 
      dir - Direction 0 = Back, 1 = Foreward
    Output -
      new_date - date of a Tuesday   
    '''
    #print('Passed date: ',origdate, 'Day of week: ',origdate.weekday())

    # # Go back to the last Tuesday
    if dir == 0 :
      if origdate.weekday() == 0 :
         new_date = origdate.date()  - timedelta(days=6)
      else: 
        new_date = origdate.date()  - timedelta(days=origdate.weekday()-1)
      return new_date


    # Go FORWARD to the NEXT Tuesday
    if dir == 1 :
      if origdate.weekday() == 1 :
         new_date = origdate.date()
      elif origdate.weekday() == 0: 
         new_date = origdate.date() + timedelta(days=origdate.weekday()+1)
      else: 
         new_date = origdate.date() + timedelta(days=8-origdate.weekday())
    return new_date


#### Standard Deviation Functions


*   stress_avgSTD
*   stress_score_avg



In [ ]:
def stress_avgSTD(userid,enddate,daysback=7):
    '''
    This function takes a datetime value and does back number of days
    then calculates the std devation of the values in the 'stress_avg'column
    Input -
      userid   - string - user_id of row passed
      enddate  - datetime - date of the row passed
      daysback - integer number of days to go back- default = 7 days
    Output -
      standard devation of the 'stress_avg' for the user is and date range passed
      from back number of days passed
    '''
    # set start date to number of days back
    start_date = enddate -  pd.to_timedelta(daysback-1, unit='d')
    # Cal std dev for the date raneg
    svstd = np.std(df_merged[(df_merged.date.between(start_date, enddate)) & (df_merged.user_id == userid)]['stress_avg'])    
   
    # #TESTING CODE
    # print('User_id: ', userid,'Start_date: ',start_date, ' End Date: ',enddate,' Num days: ',daysback)
    # xx = df_merged[df_merged.date.between(start_date, enddate)]['stress_avg']
    # print('values')
    # print(xx)
    # print('StdDev: ',svstd)
    
    # take into consideration of STD = 0
    if svstd == 0: svstd = .000001
    return svstd


def stress_score_avg(userid,enddate,daysback=7):
    '''
    This function takes a user id  and date
    then calculates the average stress score for a number of days based on the values in the 'stress_avg'column
    Input -
      userid   - string - user_id of row passed
      enddate  - datetime - date of the row passed
      daysback - integer number of days to go back- default = 7 days
    Output -
      average - Average of the values in 'stress_avg' for the user is and date range passed
      from back number of days passed
    '''
    # set start date to number of days back
    start_date = enddate -  pd.to_timedelta(daysback-1, unit='d')
    # Cal std dev for the date raneg
    ssaverage = np.average(df_merged[(df_merged.date.between(start_date, enddate)) & (df_merged.user_id == userid)]['stress_avg'])    
   
    # #TESTING CODE
    # print('User_id: ', userid,'Start_date: ',start_date, ' End Date: ',enddate,' Num days: ',daysback)
    # xx = df_merged[df_merged.date.between(start_date, enddate)]['stress_avg']
    # print('values')
    # print(xx)
    # print('Average Stress Score: ',ssaverage)
    
    return ssaverage


#### Imputing FUNCTIONS

In [ ]:
# FUNCTIONS FOR IMPUTING VALUES
def ImpNantoMedLTEQZ(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are <= 0
      # Changes any values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing that are <= 0      
    before = np.where(df1[col_name] <=0 , 1, 0).sum()
    df1[col_name] = np.where(df1[col_name]<0, np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.where(df1[col_name] <=0 , 1, 0).sum()
    print(col_name, '- Missing LTEQ 0 before - after :', before, " - ", after) 

def Imp0toMed(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are = 0
      # Changes any values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing that are = 0  
    before = np.where(df1[col_name] == 0 , 1, 0).sum()
    df1[col_name] = np.where(df1[col_name] == 0, np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.where(df1[col_name] == 0 , 1, 0).sum()
    print(col_name, '- Missing EQ 0 before - after :', before, " - ", after)


def ImpNantoMedLTZ(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are < 0
      # Changes any values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing that are <= 0      
    before = np.where(df1[col_name] < 0 , 1, 0).sum()
    df1[col_name] = np.where(df1[col_name]<0, np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.where(df1[col_name] < 0 , 1, 0).sum()
    print(col_name, '- Missing LTEQ 0 before - after :', before, " - ", after) 


def ImpNantoMed01(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are not 0 or 1
      # Changes any values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing that are not 0 or 1      
    cond1 = df1[col_name] != 0 
    cond2 = df1[col_name] != 1
    before = np.where(cond1 & cond2, 1, 0).sum()
    df1[col_name] = np.where(df1[col_name]<0, np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.where(cond1 & cond2, 1, 0).sum()
    print(col_name, '- Missing !=1,0 before - after :', before, " - ", after)     

# ************************

def ImpNoneto0(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are NaN
      # Changes the NaN's to 0
      # Prints the number of Missing that are not NaN's
    before = np.where(df1[col_name] ==' None', 1, 0).sum()
    df1[col_name] = np.where(df1[col_name] ==' None', np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(0)
    after = np.where(df1[col_name] ==' None', 1, 0).sum()
    print(col_name, '- Missing None before - after :', before, " - ", after)   



def ImpNanto0(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are NaN
      # Changes the NaN's to 0
      # Prints the number of Missing that are not NaN's
    before = np.where(df1[col_name].isna().sum())
    df1[col_name] = df1[col_name].fillna(0)
    after = np.where(df1[col_name].isna().sum())
    print(col_name, '- Missing NaN before - after :', before, " - ", after)   

 

def ImpNantoint(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are not a number
      # Changes any values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing that are not a number 
    before = np.issubdtype(df1[col_name].dtype, np.number)        
    df1[col_name] = np.where(np.issubdtype(df1[col_name].dtype, np.number), df1[col_name], np.NaN)
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.issubdtype(df1[col_name].dtype, np.number)
    print(col_name, '- Not Missing all numeric before - after :', before, " - ", after)  


def ImpText0toMed(df1,col_name):
    # This Function does the following for the column passed
      # Prints the number of Missing that are not a number
      # Changes any values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing that are not a number 
    before = np.where(df1[col_name] == 0 , 1, 0).sum()
    df1[col_name] = np.where(df1[col_name]<0, np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.where(df1[col_name] == 0 , 1, 0).sum()
    print(col_name, '- Missing EQ 0 before - after :', before, " - ", after)      


def ImpinftoMed(df1,col_name):
    # This Function does the following for the column passed
      # set before to the number that are inf
      # Changes any inf values missing to NaN
      # Changes the NaN's to the Median value 
      # Prints the number of Missing before and after 
    before = np.where(df1[col_name] == np.inf , 1, 0).sum()
    df1[col_name] = np.where(df1[col_name] == np.inf, np.NaN, df1[col_name])
    df1[col_name] = df1[col_name].fillna(df1[col_name].median())
    after = np.where(df1[col_name] == np.inf , 1, 0).sum()
    print(col_name, '- inf before - after :', before, " - ", after)      


#### Other Functions

In [ ]:
def function_missing(df):
    # Count NaN's
    list_nan = df.isna().sum()

    # Percentage of missing values for each column (To see the amount of usable data for each column and infer the effect replacing NaN's will have by some metric in part 2)
    list_nan_pct = df.isna().mean().round(4) * 100

    # Calculate the sum of the number of times a zero appears in a feature 
    list_zeros = df.isin([0]).sum()

    # Calculate a % of the number of times a zero appears in a feature
    list_zeros_pct = df.isin([0]).mean().round(4) * 100

    # create a dataframe showing the missing and percentage that is missing
    df_combined = pd.concat([list_nan.rename('# NaNs'), list_zeros.rename('# Zeros'), list_nan_pct.rename('% NaNs'), list_zeros_pct.rename('% Zeros')], axis=1)
    
    return(df_combined)

def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns    

# Correlation function
def correlation(dataset, threshold,delflag=0):
    '''
    This function takes a data frame and removes any column that is about the corrleation threshold
    If anything but a 0 is passed in delflag then no columns will be deleted 
    and you will get a list back of the column that would have been removes
    Input 
    - dataset - pandas dataframe (Default none)
    - threshold - real - the level of any coorelation above will be deleted should be a decimal
    - delflag - integer - anything other than 0 will prevent the columns from being removed (Default = 0)
    '''
    removed_list = []
    df = pd.DataFrame()
    bnumcol = len(dataset.columns)
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr().abs()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    removed_list.append(colname)
                    if delflag==0: del dataset[colname] # deleting the column from the dataset
    # print out the result
    df['deleted columns'] = removed_list
    print('Number of columns before :', bnumcol, "then after :" ,len(dataset.columns)," with a threshold of: ",threshold)
    print(df)
    print(dataset)


def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop    


## 3.1 Load CSV into a Pandas dataframe
This section is to load the original dataset from a CSV file into a dataframe


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 
%%bash
ln -s drive/My\ Drive/BUAN\ 6590\ -\ Capstone/ Capstone

ln: failed to create symbolic link 'Capstone/BUAN 6590 - Capstone': File exists


In [ ]:
# Import the original data from a excel to Pandas Data frame
df_activity = pd.read_csv('/content/Capstone/DATA/df_activity_orig.csv')
#df_activity = pd.read_csv('/content/Capstone/DATA/df_activity_new.csv',index_col=0)
df_body_battery = pd.read_csv('/content/Capstone/DATA/df_body_battery_orig.csv')
df_metrics = pd.read_csv('/content/Capstone/DATA/df_metrics_orig.csv')
df_sleep = pd.read_csv('/content/Capstone/DATA/df_sleep_orig.csv')
df_registration = pd.read_excel('/content/Capstone/DATA/PT Health Study  Registrations_orig.xlsx', sheet_name='Form Responses 1')
df_baseline = pd.read_excel('/content/Capstone/DATA/PT Health Study Baseline Survey_orig.xlsx', sheet_name='Form Responses 1')
df_survey = pd.read_excel('/content/Capstone/DATA/PT Health Study Weekly Survey_orig.xlsx', sheet_name='Form Responses 1')

## 3.2 Clean The Data <a class="anchor" id="Cleansethedata"></a>
This task involves raise the data quality to the level required by the analysis techniques that you've selected. This may involve selecting clean subsets of the data, the insertion of suitable defaults, or more ambitious techniques such as the estimation of missing data by modelling.

### Remove Duplicate Rows From Data Sets
* Activity
* Body Battery
* Metrics
* Sleep Records
* Registration
* Survey

In [ ]:
# Drop Duplicate Rows in activity
df_activity.drop_duplicates(subset=['user_id', 'date'],inplace=True)
# Drop Duplicate Rows in Body Battery
df_body_battery.drop_duplicates(subset=['user_id', 'date'],inplace=True)
# Drop Duplicate Rows in Results Metrics
df_metrics.drop_duplicates(subset=['user_id', 'date'],inplace=True)
# Drop Duplicate Rows in Sleep Records
df_sleep.drop_duplicates(subset=['user_id', 'date'],inplace=True)
# Drop Duplicate Rows in Registration records
df_registration.drop_duplicates(subset=['Client ID'],inplace=True)

##### Function Expand Activity records (HR/Stress)

In [ ]:
def splitHRandStress(df_act):
    '''
    Copys the passed Activity dataframe and then Expands and adds the heart rate columns and stress columns in the new dataframe
    Input 
    - df_act - df_activity dataframe
    Output
    - Return a new activity dataframe'
    '''  
    df_temp = df_act.copy(deep=True)

    df_temp['hr_min'] = df_act.heart_rate.str.split(pat=":|,|\n|}", n=- 1,expand=True)[1].values
    df_temp['hr_max'] = df_act.heart_rate.str.split(pat=":|,|\n|}", n=- 1,expand=True)[3].values
    df_temp['hr_res'] = df_act.heart_rate.str.split(pat=":|,|\n|}", n=- 1,expand=True)[5].values
    df_temp['stress_avg']          = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[2].values
    df_temp['stress_qualifier']    = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[4].values
    df_temp['stress_dur_rest']     = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[8].values
    df_temp['stress_dur_activity'] = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[10].values
    df_temp['stress_dur_low']      = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[12].values
    df_temp['stress_dur_medium']   = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[14].values
    df_temp['stress_dur_high']     = df_act.stress.str.split(pat=":|,|\n|}|{", n=- 1,expand=True)[16].values

    return df_temp


In [ ]:
df_activity = splitHRandStress(df_activity)

In [ ]:
# outPath = '/content/Capstone/DATA/df_activity_new.csv'
# df_activity.to_csv(outPath)

### ACTIVITY Records
* Expand Columns
* Remove Duplicate Rows
* Remove Unused Columns

####  Activity - Expand column containing more than one metric
* Activity records 
  - Heart rate - {'minimum' 'maximum', 'resting'}
  - Stress Rate - {'average', 'qualifier', 'duration'{'rest', 'activity', 'low', 'medium', 'high'}}

#### Activity - Drop Unused Columns


*   heart_rate
*   stress
*   entry_time
*   stress_qualifier



In [ ]:
# Drop unused columns in Actiity
activity_drop_col = ['heart_rate','stress','entry_time','stress_qualifier']
df_activity = df_activity.drop(activity_drop_col, axis=1)


In [ ]:
print(df_activity.shape)

(172197, 16)


### BODY BATTERY Records
* Remove Duplicate Rows
* Remove Rename Columns

#### Body Battery - Rename columns

In [ ]:
# Rename columns in Body Battery
body_battery_rename_col = {'minimum':'bb_minimum','maximum':'bb_maximum','morning':'bb_morning','midday':'bb_midday','evening':'bb_evening'}
df_body_battery.rename(columns = body_battery_rename_col, inplace = True)

## 3.3 Merge the data sets

* Activity and Body Battery
* Activity and Results Metrics
* Activity and Sleep Records
* Activity Registration Survey

### Merge Body Battery

In [ ]:
bb_merge_cols = ['user_id','date','bb_minimum','bb_maximum','bb_morning','bb_midday','bb_evening']
df_merge1 = pd.merge(df_activity, df_body_battery[bb_merge_cols], how = 'left', left_on=['user_id','date'], right_on = ['user_id','date'])

In [ ]:
df_merge1.shape

(172197, 21)

### Merge Results Metrics

In [ ]:
rm_merge_cols = ['user_id','date','vo2_max','fitness_age']
df_merge1 = pd.merge(df_merge1,df_metrics[rm_merge_cols], how = 'left', left_on=['user_id','date'], right_on = ['user_id','date'])

In [ ]:
df_merge1.shape

(172197, 23)

### Merge Sleep

In [ ]:
sl_merge_cols = ['user_id','date','total_hours','quality_hours','spo2_minimum','spo2_average','deep_hours','rem_hours']
df_merge1 = pd.merge(df_merge1, df_sleep[sl_merge_cols], how = 'left', left_on=['user_id','date'], right_on = ['user_id','date'])

In [ ]:
df_merge1.shape

(172197, 29)

### Merge Registration

In [ ]:
reg_merge_cols = ['Client ID','Age','Sex']
df_merge1 = pd.merge(df_merge1, df_registration[reg_merge_cols], how = 'left', left_on=['user_id'], right_on = ['Client ID'])

In [ ]:
df_merge1.shape

(172197, 32)

### Revew Merged Data Frame

In [ ]:
df_merged.head()

,date,duration,user_id,steps,floors,intensity_minutes,active_kilocalories,hr_min,hr_max,hr_res,stress_avg,stress_dur_rest,stress_dur_activity,stress_dur_low,stress_dur_medium,stress_dur_high,bb_minimum,bb_maximum,bb_morning,bb_midday,bb_evening,vo2_max,fitness_age,total_hours,quality_hours,spo2_minimum,spo2_average,deep_hours,rem_hours,Age,Sex,survey_date
54,2021-01-01,86400,03CimLmVHmQ0a7zj90fp,6581,3.0,286,1520,40,141,48,35.0,23280,15600,14820,8700,4140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.92,0.52,NaN,NaN,0.52,0.0,NaN,NaN,2021-01-05
55,2021-01-10,86400,03CimLmVHmQ0a7zj90fp,1898,0.0,0,58,35,100,45,19.0,34080,4380,4080,3240,720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.60,2.07,NaN,NaN,2.07,0.0,NaN,NaN,2021-01-12
56,2021-01-11,86400,03CimLmVHmQ0a7zj90fp,8297,25.0,61,1246,41,143,45,30.0,32220,9960,15840,8760,2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.63,1.15,NaN,NaN,1.15,0.0,NaN,NaN,2021-01-12
57,2021-01-12,86400,03CimLmVHmQ0a7zj90fp,5721,22.0,41,1181,38,147,44,27.0,35460,8220,12300,6780,3720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.83,1.25,NaN,NaN,1.25,0.0,NaN,NaN,2021-01-12
58,2021-01-13,86400,03CimLmVHmQ0a7zj90fp,10690,33.0,89,531,36,169,45,18.0,43560,10740,6120,3180,360,NaN,NaN,NaN,NaN,NaN,48.0,20.0,8.43,0.45,NaN,NaN,0.45,0.0,NaN,NaN,2021-01-19


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110441 entries, 54 to 172196
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   date                 110441 non-null  datetime64[ns]
 1   duration             110441 non-null  int64         
 2   user_id              110441 non-null  object        
 3   steps                110441 non-null  int64         
 4   floors               103716 non-null  float64       
 5   intensity_minutes    110441 non-null  int64         
 6   active_kilocalories  110441 non-null  int64         
 7   hr_min               110441 non-null  object        
 8   hr_max               110441 non-null  object        
 9   hr_res               110441 non-null  object        
 10  stress_avg           110441 non-null  float64       
 11  stress_dur_rest      110441 non-null  object        
 12  stress_dur_activity  110441 non-null  object        
 13  stress_dur_lo

#### TEMP Merge and Sort in user_id, date

In [ ]:
# Sort 
df_merge1.sort_values(by=['user_id', 'date'],inplace=True)
# Reindex
df_merge1.set_index(['user_id', 'date'])
df_merge1.reset_index(inplace=True)
# Drop index column
df_merge1.drop(columns=['index'],inplace=True)

#### Write out the Merged Dataframe

In [ ]:
# # Write out the merged Dataframe for use in other workbooks 
# outPath = '/content/Capstone/DATA/df_merged_data.csv'
# df_merged.to_csv(outPath)

## 3.4 Clean Merged Data Set

### 3.4.1 Drop unused columns in Merge and Sort in user_id, date

In [ ]:
# DROP columns in Merged
merge1_drop_col = ['Client ID']
df_merge1 = df_merge1.drop(merge1_drop_col, axis=1)
# Sort 
df_merge1.sort_values(by=['user_id', 'date'],inplace=True)
# Reindex
df_merge1.set_index(['user_id', 'date'])
df_merge1.reset_index(inplace=True)
# Drop index column
df_merge1.drop(columns=['index'],inplace=True)

### Additional Clean up on Merged Dataframe
* Change the date data type from object to datetime
* Drop any dates not in date rage of Weekly Survey data
  - survey dates are 5/19/2020 Last date is 4/10/2020 so only keep daily records from 5/13/2020 – 4/6/2021
* Add survey date as next Tuesday. Will be used to merge with target data set
* Drop any 'stress_avg' rows with  'None' (19,448)

In [ ]:
df_merge1.drop(df_merge1[ df_merge1['stress_avg'] == ' None'].index,inplace=True)
print(print(np.where(df_merge1['stress_avg'] == ' None',1,0).sum()))

0
None


In [ ]:
# convert date column to datetime pandas
df_merge1['date'] = pd.to_datetime(df_merge1['date'])
# convert stress_avg column to float 
df_merge1['stress_avg'] = df_merge1['stress_avg'].astype(float)  


In [ ]:
start_date = pd.to_datetime('5/13/2020 0:00')
end_date = pd.to_datetime('4/6/2021 23:59')
df_merged = df_merge1[df_merge1.date.between(start_date, end_date)].copy()

In [ ]:
# Add survey_date to match up with Target later
df_merged['survey_date'] = df_merged['date'].apply(lambda row : getTuesdaydate(row,1))

### 3.4.2 Drop Columns with a High Missing Rate <a class="anchor" id="DropCols"></a>
The following columns will be dropped because there have a high rate of missing 

* Body Battery fields (not everyoe keep their device on at night)
1.   bb_minimum
2.   bb_maximum
3.   bb_morning
4.   bb_midday
5.   bb_evening

* Missing Greater then 50%
6.   vo2_max (79%)
7.   fitness_age(79%)
8.   sex (51%)


In [ ]:
# Drop the 7 unused Columns
del_col_list = ['bb_minimum','bb_maximum','bb_morning','bb_midday','bb_evening','vo2_max','fitness_age','Sex']
df_merged.drop(del_col_list, axis=1, inplace=True)

In [ ]:
df_working = df_merged.copy(deep=True)
df_working.head()

,date,duration,user_id,steps,floors,intensity_minutes,active_kilocalories,hr_min,hr_max,hr_res,stress_avg,stress_dur_rest,stress_dur_activity,stress_dur_low,stress_dur_medium,stress_dur_high,total_hours,quality_hours,spo2_minimum,spo2_average,deep_hours,rem_hours,Age,survey_date
54,2021-01-01,86400,03CimLmVHmQ0a7zj90fp,6581,3.0,286,1520,40,141,48,35.0,23280,15600,14820,8700,4140,6.92,0.52,NaN,NaN,0.52,0.0,NaN,2021-01-05
55,2021-01-10,86400,03CimLmVHmQ0a7zj90fp,1898,0.0,0,58,35,100,45,19.0,34080,4380,4080,3240,720,9.60,2.07,NaN,NaN,2.07,0.0,NaN,2021-01-12
56,2021-01-11,86400,03CimLmVHmQ0a7zj90fp,8297,25.0,61,1246,41,143,45,30.0,32220,9960,15840,8760,2100,7.63,1.15,NaN,NaN,1.15,0.0,NaN,2021-01-12
57,2021-01-12,86400,03CimLmVHmQ0a7zj90fp,5721,22.0,41,1181,38,147,44,27.0,35460,8220,12300,6780,3720,7.83,1.25,NaN,NaN,1.25,0.0,NaN,2021-01-12
58,2021-01-13,86400,03CimLmVHmQ0a7zj90fp,10690,33.0,89,531,36,169,45,18.0,43560,10740,6120,3180,360,8.43,0.45,NaN,NaN,0.45,0.0,NaN,2021-01-19


### 3.4.3 Imputing Missing values <a class="anchor" id="Imputing Missing values"></a>

Data Missing Rules:

The following are the rules used to determine is a column is Missing and the approach to fix

1. Some integers must be Positive - 8 Found
  If an integer is negative then set to Meduim
  * total_hours
  * quality_hours
  * spo2_minimum
  * spo2_average
  * deep_hours
  * rem_hours
  * Age
2. Nan's to Zero
  * floors
3. None to Zero
  * hr_min
  * hr_max
  * hr_res


In [ ]:
function_missing(df_working)

,# NaNs,# Zeros,% NaNs,% Zeros
date,0,0,0.00,0.00
duration,0,0,0.00,0.00
user_id,0,0,0.00,0.00
steps,0,397,0.00,0.36
floors,6725,12758,6.09,11.55
intensity_minutes,0,58581,0.00,53.04
active_kilocalories,0,631,0.00,0.57
hr_min,0,0,0.00,0.00
hr_max,0,0,0.00,0.00
hr_res,0,0,0.00,0.00


In [ ]:
### Only Missing vales to Medium
ImpText0toMed(df_working,'total_hours')
ImpText0toMed(df_working,'quality_hours')
ImpText0toMed(df_working,'spo2_minimum')
ImpText0toMed(df_working,'spo2_average')
ImpText0toMed(df_working,'deep_hours')
ImpText0toMed(df_working,'rem_hours')
ImpText0toMed(df_working,'Age')
# Nan's to Zero
ImpNanto0(df_working,'floors')
# None to Zero
ImpNoneto0(df_working,'hr_min')
ImpNoneto0(df_working,'hr_max')
ImpNoneto0(df_working,'hr_res')

total_hours - Missing EQ 0 before - after : 1  -  1
quality_hours - Missing EQ 0 before - after : 143  -  143
spo2_minimum - Missing EQ 0 before - after : 0  -  0
spo2_average - Missing EQ 0 before - after : 0  -  0
deep_hours - Missing EQ 0 before - after : 1319  -  1319
rem_hours - Missing EQ 0 before - after : 12669  -  12669
Age - Missing EQ 0 before - after : 0  -  0
floors - Missing NaN before - after : (array([0]),)  -  (array([], dtype=int64),)
hr_min - Missing None before - after : 25  -  0
hr_max - Missing None before - after : 25  -  0
hr_res - Missing None before - after : 22  -  0


### 3.4.4 Convert Object data types to integer

The following are the columns data types were changed from object to Integers

* hr_min
* hr_max
* hr_res
* stress_dur_rest    
* stress_dur_activity 
* stress_dur_low   
* stress_dur_medium
* stress_dur_high

In [ ]:
# Convert Object columns to integer
df_working['hr_min'] = df_working['hr_min'].astype(float)                        
df_working['hr_max'] = df_working['hr_max'].astype(float)                        
df_working['hr_res'] = df_working['hr_res'].astype(float)                        
df_working['stress_dur_rest'] = df_working['stress_dur_rest'].astype(float)               
df_working['stress_dur_activity'] = df_working['stress_dur_activity'].astype(float)           
df_working['stress_dur_low'] = df_working['stress_dur_low'].astype(float)                
df_working['stress_dur_medium'] = df_working['stress_dur_medium'].astype(float)
df_working['stress_dur_high'] = df_working['stress_dur_high'].astype(float)

In [ ]:
# Write out the merged Dataframe for use in other workbooks 
outPath = '/content/Capstone/DATA/df_working_beforeCalc.csv'
df_working.to_csv(outPath)

## 3.5  Add Additional Predictors


### 3.5.1 Add Calculated predictors


*	7-day Stress score moving average
*	28-day Stress score moving average
*	28-day Stress score STD Deviation
*	7 Day Stress Score Moving Average within 28-day Stress Score Moving average + 0 to 1 STD DEV
*	7 Day Stress Score Moving Average within 28-day Stress Score Moving average + 1 to 2 STD DEV
*	7 Day Stress Score Moving Average within 28-day Stress Score Moving average + 2 to 3 STD DEV
*	7 Day Stress Score Moving Average within 28-day Stress Score Moving average + > 3 STD DEV List item
*	Number of STD Deviation 7 Day Stress score id from 28day Stress score average



In [ ]:
# 7 day and 28 day stress score moving average
df_working['ss7dmavg']  = df_working.apply(lambda row : stress_score_avg( row['user_id'],row['date'],7), axis=1)
df_working['ss28dmavg'] = df_working.apply(lambda row : stress_score_avg( row['user_id'],row['date'],28), axis=1)

In [ ]:
# 28 day Stress score STD Devation
df_working['ss28dStdev'] = df_working.apply(lambda row : stress_avgSTD( row['user_id'],row['date'],28), axis=1)

In [ ]:
# 7DSSMA0-1STDDev	7 Day Stress Score Moving Average within 28 day Stress Score Moving average + 0 to 1 STD DEV
df_working['7Dssma0-1STDev']  =  np.where(  
      ((df_working['ss7dmavg'] - df_working['ss28dmavg'])  > 0)  &
       (df_working['ss7dmavg'] >= df_working['ss28dmavg'])                                          &
       (df_working['ss7dmavg'] <= (df_working['ss28dmavg']) + (1 * df_working['ss28dStdev'])), 'True', 'False' )

# 7DSSMA1-2STDDev	7 Day Stress Score Moving Average within 28 day Stress Score Moving average + 1 to 2 STD DEV
df_working['7Dssma1-2STDev']  =  np.where( 
      ((df_working['ss7dmavg'] - df_working['ss28dmavg'])  > 0)  &
       (df_working['ss7dmavg'] >= (df_working['ss28dmavg'] + (1 * df_working['ss28dStdev'])))        &
       (df_working['ss7dmavg'] <= (df_working['ss28dmavg'] + (2 * df_working['ss28dStdev']))), 'True', 'False' )

# # # 7DSSMA2-3STDDev	7 Day Stress Score Moving Average within 28 day Stress Score Moving average + 2 to 3 STD DEV
df_working['7Dssma2-3STDev']  =  np.where( 
      ((df_working['ss7dmavg'] - df_working['ss28dmavg'])  > 0)  &
       (df_working['ss7dmavg'] >= (df_working['ss28dmavg'] + (2 * df_working['ss28dStdev'])))        &
       (df_working['ss7dmavg'] <= (df_working['ss28dmavg'] + (3 * df_working['ss28dStdev']))), 'True', 'False' )

# # 7DSSMA3+STDDev	7 Day Stress Score Moving Average within 28 day Stress Score Moving average + > 3 STD DEV
df_working['7Dssma3+STDev']  =  np.where( 
      ((df_working['ss7dmavg'] - df_working['ss28dmavg'])  > 0)  &
       (df_working['ss7dmavg'] >= (df_working['ss28dmavg'] + (3 * df_working['ss28dStdev']))), 'True', 'False' )



In [ ]:
# # For testing only

# df_working['7d-18Plus1std']   =  df_working['ss7dmavg']  - (df_working['ss28dmavg'] +(1 * df_working['ss28dStdev']))
# df_working['1std'] =  (df_working['ss28dmavg'] + (1 * df_working['ss28dStdev']))
# df_working['2std'] =  (df_working['ss28dmavg'] + (2 * df_working['ss28dStdev']))
# df_working['3std'] =  (df_working['ss28dmavg'] + (3 * df_working['ss28dStdev']))

In [ ]:
# Number of STD Deviation 7 Day Stress score id from 28day Stress score average
df_working['7D_StdDevfrom28d']  = (df_working['ss7dmavg'] - df_working['ss28dmavg'])/df_working['ss28dStdev']

### 3.5.2 One Hot Encode columns
* 7Dssma0-1STDev
* 7Dssma1-2STDev
* 7Dssma2-3STDev
* 7Dssma3+STDev

### 3.5.3 Review Final Merge and Sort in user_id, date

In [ ]:
# Sort 
df_merge1.sort_values(by=['user_id', 'date'],inplace=True)
# Reindex
df_merge1.set_index(['user_id', 'date'])
df_merge1.reset_index(inplace=True)
# Drop index column
df_merge1.drop(columns=['index'],inplace=True)

#### Write out the Merged Dataframe

In [ ]:
# # Write out the merged Dataframe for use in other workbooks 
# outPath = '/content/Capstone/DATA/df_merged_data.csv'
# df_merged.to_csv(outPath)

#### Review Merged Data Frame

In [ ]:
df_merged.head()

,date,duration,user_id,steps,floors,intensity_minutes,active_kilocalories,hr_min,hr_max,hr_res,stress_avg,stress_dur_rest,stress_dur_activity,stress_dur_low,stress_dur_medium,stress_dur_high,bb_minimum,bb_maximum,bb_morning,bb_midday,bb_evening,vo2_max,fitness_age,total_hours,quality_hours,spo2_minimum,spo2_average,deep_hours,rem_hours,Age,Sex,survey_date
54,2021-01-01,86400,03CimLmVHmQ0a7zj90fp,6581,3.0,286,1520,40,141,48,35.0,23280,15600,14820,8700,4140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.92,0.52,NaN,NaN,0.52,0.0,NaN,NaN,2021-01-05
55,2021-01-10,86400,03CimLmVHmQ0a7zj90fp,1898,0.0,0,58,35,100,45,19.0,34080,4380,4080,3240,720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.60,2.07,NaN,NaN,2.07,0.0,NaN,NaN,2021-01-12
56,2021-01-11,86400,03CimLmVHmQ0a7zj90fp,8297,25.0,61,1246,41,143,45,30.0,32220,9960,15840,8760,2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.63,1.15,NaN,NaN,1.15,0.0,NaN,NaN,2021-01-12
57,2021-01-12,86400,03CimLmVHmQ0a7zj90fp,5721,22.0,41,1181,38,147,44,27.0,35460,8220,12300,6780,3720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.83,1.25,NaN,NaN,1.25,0.0,NaN,NaN,2021-01-12
58,2021-01-13,86400,03CimLmVHmQ0a7zj90fp,10690,33.0,89,531,36,169,45,18.0,43560,10740,6120,3180,360,NaN,NaN,NaN,NaN,NaN,48.0,20.0,8.43,0.45,NaN,NaN,0.45,0.0,NaN,NaN,2021-01-19


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110441 entries, 54 to 172196
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   date                 110441 non-null  datetime64[ns]
 1   duration             110441 non-null  int64         
 2   user_id              110441 non-null  object        
 3   steps                110441 non-null  int64         
 4   floors               103716 non-null  float64       
 5   intensity_minutes    110441 non-null  int64         
 6   active_kilocalories  110441 non-null  int64         
 7   hr_min               110441 non-null  object        
 8   hr_max               110441 non-null  object        
 9   hr_res               110441 non-null  object        
 10  stress_avg           110441 non-null  float64       
 11  stress_dur_rest      110441 non-null  object        
 12  stress_dur_activity  110441 non-null  object        
 13  stress_dur_lo

#### Automated EDA Python package - SWEETVIZ

In [ ]:
# !pip install sweetviz

In [ ]:
# import sweetviz

#### Sweetviz Report

In [ ]:
# my_report = sweetviz.analyze(df_merged)
# my_report.show_notebook()

In [ ]:
# Multiple categorical columns
one_hot_cols = ['7Dssma0-1STDev','7Dssma1-2STDev','7Dssma2-3STDev','7Dssma3+STDev']
df_working = pd.get_dummies(df_working, columns=one_hot_cols)

In [ ]:
# Sort temp_merged
df_working.sort_values(by=['user_id', 'date'],inplace=True)
# Reindex
df_working.set_index(['user_id', 'date'])
df_working.reset_index(inplace=True)
# Drop index column
df_working.drop(columns=['index'],inplace=True)

In [ ]:
# temp_merged
print(df_working.shape)
print(df_working.info())
#V.head()
df_working.describe().T

(110441, 36)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110441 entries, 0 to 110440
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   date                  110441 non-null  datetime64[ns]
 1   duration              110441 non-null  int64         
 2   user_id               110441 non-null  object        
 3   steps                 110441 non-null  int64         
 4   floors                110441 non-null  float64       
 5   intensity_minutes     110441 non-null  int64         
 6   active_kilocalories   110441 non-null  int64         
 7   hr_min                110441 non-null  float64       
 8   hr_max                110441 non-null  float64       
 9   hr_res                110441 non-null  float64       
 10  stress_avg            110441 non-null  float64       
 11  stress_dur_rest       110441 non-null  float64       
 12  stress_dur_activity   110441 non-null  float6

,count,mean,std,min,25%,50%,75%,max
duration,110441.0,86415.325830,285.080841,86400.000000,86400.000000,86400.000000,86400.000000,104400.000000
steps,110441.0,8457.848625,5306.904453,0.000000,4891.000000,7562.000000,10882.000000,144918.000000
floors,110441.0,11.579214,21.016327,0.000000,2.000000,6.000000,14.000000,1896.000000
intensity_minutes,110441.0,38.434712,67.746505,0.000000,0.000000,0.000000,58.000000,2044.000000
active_kilocalories,110441.0,548.474923,440.243641,0.000000,243.000000,453.000000,733.000000,10396.000000
hr_min,110441.0,50.608461,7.720848,0.000000,45.000000,50.000000,55.000000,121.000000
hr_max,110441.0,131.051973,23.477596,0.000000,114.000000,125.000000,148.000000,243.000000
hr_res,110441.0,57.041054,8.219690,0.000000,51.000000,57.000000,62.000000,121.000000
stress_avg,110441.0,30.074637,10.535080,1.000000,23.000000,29.000000,36.000000,99.000000
stress_dur_rest,110441.0,29028.644978,13190.964693,0.000000,20520.000000,29340.000000,37680.000000,78660.000000


In [ ]:
function_missing(df_working)

,# NaNs,# Zeros,% NaNs,% Zeros
date,0,0,0.0,0.00
duration,0,0,0.0,0.00
user_id,0,0,0.0,0.00
steps,0,397,0.0,0.36
floors,0,19483,0.0,17.64
intensity_minutes,0,58581,0.0,53.04
active_kilocalories,0,631,0.0,0.57
hr_min,0,25,0.0,0.02
hr_max,0,25,0.0,0.02
hr_res,0,22,0.0,0.02


In [ ]:
df_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110441 entries, 0 to 110440
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   date                  110441 non-null  datetime64[ns]
 1   duration              110441 non-null  int64         
 2   user_id               110441 non-null  object        
 3   steps                 110441 non-null  int64         
 4   floors                110441 non-null  float64       
 5   intensity_minutes     110441 non-null  int64         
 6   active_kilocalories   110441 non-null  int64         
 7   hr_min                110441 non-null  float64       
 8   hr_max                110441 non-null  float64       
 9   hr_res                110441 non-null  float64       
 10  stress_avg            110441 non-null  float64       
 11  stress_dur_rest       110441 non-null  float64       
 12  stress_dur_activity   110441 non-null  float64       
 13 

In [ ]:
# Index clean up
# Sort 
df_working.sort_values(by=['user_id', 'date'],inplace=True)
# Reindex
df_working.set_index(['user_id', 'date'])
df_working.reset_index(inplace=True)
# Drop index column
df_working.drop(columns=['index'],inplace=True)

In [ ]:
# Write out the merged Dataframe for use in other workbooks 
outPath = '/content/Capstone/DATA/df_merged_clean.csv'
df_working.to_csv(outPath)

## 3.6 CLEAN UP SURVEY DATA AND CREATE TARGET
This task includes constructive data preparation operations such as the production of derived attributes (Status)

#### Target Field "Status" Decode table


####  

| Oiginal value | New Value |
|-|-|
| About the same | Same |
| Better than usual | Better |
| I felt over-stressed and/or   under-recovered | Stress |
| I had a flare up of a chronic   condition | Unwell |
| I had a new onset of illness | Unwell |
| I had increased pain or an   injury | Unwell |
| Significantly Worse, in   distress | Unwell |

### 3.6.1 Add New Columns, Change survey_date from object to datetime, Remove duplicates
* 'Status'      - new target field
* 'survey_date' - new survey date field
* Change survey_date to a datetime
* Remove Duplicate records 

In [ ]:
# Create new Status filed (Target)
func = np.vectorize(statusconditions)
df_survey['Status'] = func(df_survey['Current Status'])

# Set the new column survey_date to make all the dates consistant to Tuesday
df_survey['survey_date'] = df_survey['Timestamp'].apply(lambda row : getTuesdaydate(row,0))

# convert survey_date column to datetime 
df_survey['survey_date'] = pd.to_datetime(df_survey['survey_date'])    
## MOVE UP
# convert date column to datetime pandas
df_working['survey_date'] = pd.to_datetime(df_working['survey_date'])  

In [ ]:
# Drop Duplicate Rows in Survey
df_survey.drop_duplicates(subset=['Client ID', 'survey_date'],inplace=True)

In [ ]:
# Drop Rows Where Client Id is blank
df_survey.dropna(axis=0, subset=['Client ID'], inplace=True)

# Check Results
print(np.where(pd.isnull(df_survey['Client ID']) , 1, 0).sum())

0


In [ ]:
## Drop unused columns
df_survey_new = df_survey[['Client ID', 'survey_date','Status']].copy()

### 3.6.2 Rename, Sort and Reindex Weekly Survey columns 
* 'Client ID' to 'user_id'
* Sort by  'user_id', 'survey_date'
* Index by 'user_id', 'survey_date'

In [ ]:
# Rename columns in Weekly Survey
df_survey_rename_col = {'Client ID' : 'user_id'}
df_survey_new.rename(columns = df_survey_rename_col, inplace = True)


# Rename columns in Weekly Survey
df_survey_rename_col = {'user_id'    : 'user_idx',
                        'survey_date' :'survey_datex'}
df_survey_new.rename(columns = df_survey_rename_col, inplace = True)

# Sort 
df_survey_new.sort_values(by=['user_idx', 'survey_datex'],inplace=True)
# Reindex
df_survey_new.set_index(['user_idx', 'survey_datex'])
df_survey_new.reset_index(inplace=True)
# Drop index column
df_survey_new.drop(columns=['index'],inplace=True)

In [ ]:
# Reindex
df_working.set_index(['user_id', 'survey_date'])
df_working.reset_index(inplace=True)
# Drop index column
df_working.drop(columns=['index'],inplace=True)

In [ ]:
df_working.shape

(110441, 36)

In [ ]:
# ## Copy only the columns needed
survey_merge_cols = ['Status','survey_datex']
#df_merge2 = pd.merge(df_working, df_survey_new[survey_merge_cols],
df_merge2 = pd.merge(df_working, df_survey_new,
                       how = 'inner', 
                       left_on  = ['user_id' ,'survey_date'], 
                       right_on = ['user_idx','survey_datex'])

### Check for no blanks in survey

In [ ]:
# #df_survey.info()
# print(df_working.info())
# np.where(pd.isnull(df_merge2['Status']) , 1, 0).sum()
# print(df_merge2.info())


### 3.6.3 Review Final Merge2

#### Automated EDA Python package - SWEETVIZ

In [ ]:
# !pip install sweetviz

#### Sweetviz Report

In [ ]:
# my_report = sweetviz.analyze(df_merge2)
# my_report.show_notebook()

###  Write out new Survey Dataframe

In [ ]:
# Remove redunded columns
## Drop unused columns
df_merge2.drop(columns=['user_idx','survey_datex'],inplace=True)

# Sort 
df_merge2.sort_values(by=['user_id', 'survey_date'],inplace=True)
# Reindex
df_merge2.set_index(['user_id', 'survey_date'])
df_merge2.reset_index(inplace=True)
# Drop index column
df_merge2.drop(columns=['index'],inplace=True)

In [ ]:
df_merge2.info()
df_merge2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37109 entries, 0 to 37108
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  37109 non-null  datetime64[ns]
 1   duration              37109 non-null  int64         
 2   user_id               37109 non-null  object        
 3   steps                 37109 non-null  int64         
 4   floors                37109 non-null  float64       
 5   intensity_minutes     37109 non-null  int64         
 6   active_kilocalories   37109 non-null  int64         
 7   hr_min                37109 non-null  float64       
 8   hr_max                37109 non-null  float64       
 9   hr_res                37109 non-null  float64       
 10  stress_avg            37109 non-null  float64       
 11  stress_dur_rest       37109 non-null  float64       
 12  stress_dur_activity   37109 non-null  float64       
 13  stress_dur_low  

,date,duration,user_id,steps,floors,intensity_minutes,active_kilocalories,hr_min,hr_max,hr_res,stress_avg,stress_dur_rest,stress_dur_activity,stress_dur_low,stress_dur_medium,stress_dur_high,total_hours,quality_hours,spo2_minimum,spo2_average,deep_hours,rem_hours,Age,survey_date,ss7dmavg,ss28dmavg,ss28dStdev,7D_StdDevfrom28d,7Dssma0-1STDev_False,7Dssma0-1STDev_True,7Dssma1-2STDev_False,7Dssma1-2STDev_True,7Dssma2-3STDev_False,7Dssma2-3STDev_True,7Dssma3+STDev_False,7Dssma3+STDev_True,Status
0,2020-05-13,86400,0Sq4rLw6hryK3GlUpE6n,7798,0.0,0,175,42.0,131.0,52.0,37.0,22140.0,12240.0,14760.0,16200.0,4440.0,5.25,2.97,88.0,95.38,1.45,1.07,68.0,2020-05-19,37.000000,37.000000,0.000001,0.0,1,0,1,0,1,0,1,0,2
1,2020-05-14,86400,0Sq4rLw6hryK3GlUpE6n,7787,11.0,0,178,37.0,100.0,54.0,45.0,15420.0,20340.0,12720.0,19920.0,5340.0,6.45,3.53,84.0,92.06,1.45,1.07,68.0,2020-05-19,41.000000,41.000000,4.000000,0.0,1,0,1,0,1,0,1,0,2
2,2020-05-15,86400,0Sq4rLw6hryK3GlUpE6n,6432,8.0,0,134,48.0,104.0,52.0,43.0,19860.0,15660.0,13560.0,17100.0,8880.0,5.93,1.67,87.0,93.78,1.45,1.07,68.0,2020-05-19,41.666667,41.666667,3.399346,0.0,1,0,1,0,1,0,1,0,2
3,2020-05-16,86400,0Sq4rLw6hryK3GlUpE6n,6682,5.0,0,253,49.0,111.0,52.0,53.0,12600.0,15600.0,9240.0,17940.0,14040.0,7.58,3.77,84.0,93.16,1.45,1.07,68.0,2020-05-19,44.500000,44.500000,5.722762,0.0,1,0,1,0,1,0,1,0,2
4,2020-05-17,86400,0Sq4rLw6hryK3GlUpE6n,5406,8.0,0,175,52.0,108.0,58.0,66.0,4800.0,24060.0,6000.0,11220.0,24060.0,5.60,4.02,83.0,95.79,1.45,1.07,68.0,2020-05-19,48.800000,48.800000,10.007997,0.0,1,0,1,0,1,0,1,0,2


In [ ]:
# Write out the merged Dataframe for use in other workbooks 
outPath = '/content/Capstone/DATA/df_merge2_with_Target.csv'
df_merge2.to_csv(outPath)